In [1]:
import coax
from coax._core.q import Q
import gym
import jax.numpy as jnp

import edesdetectrl.environments.binary_classification
from edesdetectrl.config import config
import edesdetectrl.model as model

/Users/magnus/research/edesdetect/.venv/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
env = gym.make("EDESClassification-v0")

q = coax.Q(model.get_func_approx(env), env)
q.params = coax.utils.load(config['data']['trained_params_path'])

In [13]:
done = False
s = env.reset()
all_q_values = []
all_states = []
all_rewards = []
while not done:
    q_values = q(s)
    a = jnp.argmax(q_values)
    s, r, done, info = env.step(a)

    all_q_values.append(q_values)
    all_states.append(s)
    all_rewards.append(r)


env.close()

In [21]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib widget

fig, ax = plt.subplots(ncols=2)
ax[0].plot(all_q_values)
ax[0].plot(all_rewards)
ax[0].legend(['0','1','Reward'])

im = ax[1].imshow(all_states[0][:,:,1])
def update(i):
    im.set_data(all_states[i][:,:,1])

anim = FuncAnimation(fig, update, len(all_states))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/magnus/research/edesdetect/.venv/lib/python3.8/site-packages/matplotlib/animation.py:973: UserWarning: Animation was deleted without rendering anything. This is most likely unintended. To prevent deletion, assign the Animation to a variable that exists for as long as you need the Animation.
  warnings.warn(


In [23]:
anim.pause()